In [ ]:
!pip install openai
!pip install python-dotenv

In [ ]:
import panel as pn  
import os
import openai
from dotenv import load_dotenv

load_dotenv()

openai.api_key = os.environ['OPENAI_API_KEY']



def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature,  
    )

    return response.choices[0].message["content"]


def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role': 'user', 'content': f"{prompt}"})
    response = get_completion_from_messages(context)
    context.append({'role': 'assistant', 'content': f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))

    return pn.Column(*panels)


pn.extension()

panels = []  # collect display



In [ ]:
context = [{'role': 'system', 'content': """
You are OrderBot, an automated service to collect orders for a bakery. \
You first greet the customer, then collect the order, \
and then asks if it's a pickup or delivery. \
You wait to collect the entire order, then summarize it and check for a final \
time if the customer wants to add anything else. \
Make sure to ask if the customer wants to pay with cash or credit card.\
If it's a delivery, you ask for an address. \
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a short, very conversational friendly style. \
The menu includes \
Breads: \
white bread 10.00 \
whole wheat bread   10.95 \
rye bread   11.95 \
sourdough bread 14.35 \
multigrain bread 7.25 \
baguette 5.00 \
ciabatta 4.75 \
Cakes: \
chocolate cake 25.00, \
vanilla cake 26.50 \
carrot cake 22.00 \
birthday cake 18.50 \
lemon cake 25.50 \
meringue cake 45.00 \
Cookies: \
chocolate chip cookies 1.85 \
oatmeal raisin cookies 1.50 \
gingerbread cookies 2.00 \
peanut butter cookies 2.20 \

"""}]  # accumulate messages



inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

In [ ]:
messages =  context.copy()
messages.append(
{'role':'system', 'content':'create a json summary of the previous food order. Itemize the price for each item\
 The fields in the first table should be 1) unique order id  2) total price \
 Create tables for each unique item ordered and have following fields 1) item name 2) item quantity'},    
)


response = get_completion_from_messages(messages, temperature=0)
print(response)